In [2]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [3]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [4]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 4


# Check the drug

In [5]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-03 06:47:46] ax.service.ax_client: Generated new trial 64 with parameters {'Drug': 50, 'SL_1': 0, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 100, 'S_3': 0, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the re

29 min


In [6]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.00000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0
mean,71.500000,0.990125,0.008625,0.0,0.0,0.0,0.00125,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
std,4.760952,0.034529,0.034500,0.0,0.0,0.0,0.00500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,64.000000,0.862000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
25%,67.750000,1.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
50%,71.500000,1.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
75%,75.250000,1.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
max,79.000000,1.000000,0.138000,0.0,0.0,0.0,0.02000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0000,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.0
mean,71.500000,53.125000,0.5,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.0,97.937500,0.0,0.0
std,4.760952,6.888396,2.0,0.0,0.0,0.0,0.2500,0.0,0.0,0.0,0.0,0.0,3.586433,0.0,0.0
min,64.000000,49.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,91.000000,0.0,0.0
25%,67.750000,50.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,98.250000,0.0,0.0
50%,71.500000,50.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,100.000000,0.0,0.0
75%,75.250000,50.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,100.000000,0.0,0.0
max,79.000000,67.000000,8.0,0.0,0.0,0.0,1.0000,0.0,0.0,0.0,0.0,0.0,100.000000,0.0,0.0


In [8]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.00000,16.0,16.0,16.0,16.0000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0
mean,71.500000,346.543750,3.01875,0.0,0.0,0.0,0.4375,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0
std,4.760952,12.085141,12.07500,0.0,0.0,0.0,1.7500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,64.000000,301.700000,0.00000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0
25%,67.750000,350.000000,0.00000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0
50%,71.500000,350.000000,0.00000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0
75%,75.250000,350.000000,0.00000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0
max,79.000000,350.000000,48.30000,0.0,0.0,0.0,7.0000,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0


In [9]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        5.5
SL_1        0.0
SL_2        0.0
SL_3        0.0
LL_1        0.0
LL_2        0.0
LL_3        0.0
P_1         0.0
P_2         0.0
P_3         0.0
S_1         0.0
S_2        16.0
S_3         0.0
Water       0.0
Solvent     0.0
dtype: float64

In [6]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [11]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [12]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [13]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [14]:
updated_script

('protocol/iteration_4_OT_2_protocol_1.py',
 'protocol/iteration_4_OT_2_protocol_2.py')

# Perform experiment

# Results

In [7]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,64,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0
1,65,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0
2,66,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0
3,67,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0
4,68,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0
5,69,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0
6,70,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0
7,71,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0
8,72,0.980,0.000,0,0,0,0.02,0,0,0,0,0,1,0,0,2,0
9,73,1.000,0.000,0,0,0,0.00,0,0,0,0,0,1,0,0,1,0


In [8]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,25.028681,1.666525e-01
1,Good,A2,24.431515,2.083099e-01
2,Error,A3,1908.673017,5.707963e-01
3,Caution,A4,46.783219,7.339022e-09
4,Caution,A5,24.901976,2.069841e-01
5,Caution,A6,18.266479,5.707963e-01
6,Good,B1,24.835932,2.116881e-01
7,Caution,B2,23.618391,2.331611e-01
8,Caution,B3,23.659851,2.633519e-01
9,Good,B4,24.343256,1.150438e-01


In [9]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,24.730098,0.422260,0.187481,0.029456,1
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,23.889863,0.641194,0.211722,0.136724,1
4,Formulation 5,23.491139,0.202682,0.179959,0.070910,1
5,Formulation 6,24.658707,0.867646,0.243748,0.074609,1
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,23.218624,0.163549,0.175085,0.041483,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,23.190214,0.881020,0.156765,0.074730,1


In [10]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,64,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 1,24.730098,0.422260,0.187481,0.029456,1
1,65,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,66,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,67,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 4,23.889863,0.641194,0.211722,0.136724,1
4,68,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 5,23.491139,0.202682,0.179959,0.070910,1
5,69,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 6,24.658707,0.867646,0.243748,0.074609,1
6,70,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,71,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 8,23.218624,0.163549,0.175085,0.041483,1
8,72,0.980,0.000,0,0,0,0.02,0,0,0,...,0,0,2,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,73,1.000,0.000,0,0,0,0.00,0,0,0,...,0,0,1,0,Formulation 10,23.190214,0.881020,0.156765,0.074730,1


In [11]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,64,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 1,24.730098,0.422260,0.187481,0.029456,1,2000.0,0
1,65,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,66,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,67,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 4,23.889863,0.641194,0.211722,0.136724,1,2000.0,0
4,68,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 5,23.491139,0.202682,0.179959,0.070910,1,2000.0,0
5,69,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 6,24.658707,0.867646,0.243748,0.074609,1,2000.0,0
6,70,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,71,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 8,23.218624,0.163549,0.175085,0.041483,1,2000.0,0
8,72,0.980,0.000,0,0,0,0.02,0,0,0,...,2,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,73,1.000,0.000,0,0,0,0.00,0,0,0,...,1,0,Formulation 10,23.190214,0.881020,0.156765,0.074730,1,2000.0,0


In [12]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.024730,0.000422,1.0,0.0,0.187481,0.029456,0.083333
1,0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.083333
2,0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.083333
3,1,0.023890,0.000641,1.0,0.0,0.211722,0.136724,0.083333
4,1,0.023491,0.000203,1.0,0.0,0.179959,0.070910,0.083333
5,1,0.024659,0.000868,1.0,0.0,0.243748,0.074609,0.083333
6,0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.083333
7,1,0.023219,0.000164,1.0,0.0,0.175085,0.041483,0.083333
8,0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.166667
9,1,0.023190,0.000881,1.0,0.0,0.156765,0.074730,0.083333


In [13]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.024730,0.000422,1.0,0.0,0.187481,0.029456,0.083333
1,0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.083333
2,0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.083333
3,1,0.023890,0.000641,1.0,0.0,0.211722,0.136724,0.083333
4,1,0.023491,0.000203,1.0,0.0,0.179959,0.070910,0.083333
5,1,0.024659,0.000868,1.0,0.0,0.243748,0.074609,0.083333
6,0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.083333
7,1,0.023219,0.000164,1.0,0.0,0.175085,0.041483,0.083333
8,0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.166667
9,1,0.023190,0.000881,1.0,0.0,0.156765,0.074730,0.083333


In [14]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [15]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [16]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,64,1.0,0.0,0,0,0,0.0,0,0,0,...,0.083333,0,Formulation 1,0.024730,0.000422,0.187481,0.029456,1,1,0
1,65,1.0,0.0,0,0,0,0.0,0,0,0,...,0.083333,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0,0
2,66,1.0,0.0,0,0,0,0.0,0,0,0,...,0.083333,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0,0
3,67,1.0,0.0,0,0,0,0.0,0,0,0,...,0.083333,0,Formulation 4,0.023890,0.000641,0.211722,0.136724,1,1,0
4,68,1.0,0.0,0,0,0,0.0,0,0,0,...,0.083333,0,Formulation 5,0.023491,0.000203,0.179959,0.070910,1,1,0


In [17]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [18]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [19]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-05 06:32:00] ax.service.ax_client: Completed trial 64 with data: {'Solu': (1, 0), 'Size': (0.02473, 0.000422), 'PDI': (0.187481, 0.029456)}.
[INFO 08-05 06:32:00] ax.service.ax_client: Completed trial 65 with data: {'Solu': (0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-05 06:32:00] ax.service.ax_client: Completed trial 66 with data: {'Solu': (0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-05 06:32:00] ax.service.ax_client: Completed trial 67 with data: {'Solu': (1, 0), 'Size': (0.02389, 0.000641), 'PDI': (0.211722, 0.136724)}.
[INFO 08-05 06:32:00] ax.service.ax_client: Completed trial 68 with data: {'Solu': (1, 0), 'Size': (0.023491, 0.000203), 'PDI': (0.179959, 0.07091)}.
[INFO 08-05 06:32:00] ax.service.ax_client: Completed trial 69 with data: {'Solu': (1, 0), 'Size': (0.024659, 0.000868), 'PDI': (0.243748, 0.074609)}.
[INFO 08-05 06:32:00] ax.service.ax_client: Completed trial 70 with data: {'Solu': (0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 